# The revised simplex algorithm

In [1]:
import numpy as np

import mcf_simplex_analyzer.fractionarray as fa
import mcf_simplex_analyzer.fractionarray.sparse as fas
from mcf_simplex_analyzer.simplex._revised_simplex import lu_factor, PLUSolver, EtaSolver, EtaFile

In [2]:
# Enable logging
if True:
    import logging

    logging.basicConfig(
        format="%(asctime)s %(levelname)s:%(message)s",
        datefmt="%I:%M:%S",
        level=logging.DEBUG,
    )

In [3]:
A = fa.FractionArray.from_array( np.array(
    [[3, 2, 1, 2, 1, 0, 0], [1, 1, 1, 1, 0, 1, 0], [4, 3, 3, 4, 0, 0, 1]], dtype=int
))
A = fas.dense_to_csc(A)
A

FractionCSCMatrix(shape=(3, 7), indptr=array([ 0,  3,  6,  9, 12, 13, 14, 15]), indices=array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]), data=FractionArray(data=[mpq(3,1) mpq(1,1) mpq(4,1) mpq(2,1) mpq(1,1) mpq(3,1) mpq(1,1) mpq(1,1)
 mpq(3,1) mpq(2,1) mpq(1,1) mpq(4,1) mpq(1,1) mpq(1,1) mpq(1,1)]))

In [4]:
b = fa.FractionArray.from_array(np.array([225, 117, 420], dtype=np.int64))
b

FractionArray(data=[mpq(225,1) mpq(117,1) mpq(420,1)])

In [5]:
c = fa.FractionArray.from_array(np.array([19, 13, 12, 17, 0, 0, 0], dtype=np.int64))
c

FractionArray(data=[mpq(19,1) mpq(13,1) mpq(12,1) mpq(17,1) mpq(0,1) mpq(0,1) mpq(0,1)])

In [6]:
base = np.zeros(A.shape[1], dtype=bool)
base[[0, 2, 6]] = True
base

array([ True, False,  True, False, False, False,  True])

In [7]:
base_ind = np.where(base)[0]
base_ind

array([0, 2, 6])

In [8]:
def csc_select_columns(csc: fas.FractionCSCMatrix, columns: np.ndarray):
    m, _ = csc.shape
    n = len(columns)
    
    out = fa.zeros((m, n))
    for i, col in enumerate(columns):
        start, end = csc.indptr[col], csc.indptr[col + 1]
        out[csc.indices[start:end], i] = csc.data[start:end]
                
    return out

In [9]:
csc_select_columns(A, base_ind)

FractionArray(data=[[mpq(3,1) mpq(1,1) mpq(0,1)]
 [mpq(1,1) mpq(1,1) mpq(0,1)]
 [mpq(4,1) mpq(3,1) mpq(1,1)]])

In [10]:
B = fas.csc_to_dense(fas.csc_select_columns(A, base_ind))
print(B)

perm, lu = lu_factor(B)
print(lu)

solver = PLUSolver(lu=lu, perm=perm)
solver

FractionArray(data=[[mpq(3,1) mpq(1,1) mpq(0,1)]
 [mpq(1,1) mpq(1,1) mpq(0,1)]
 [mpq(4,1) mpq(3,1) mpq(1,1)]])
FractionArray(data=[[mpq(3,1) mpq(1,1) mpq(0,1)]
 [mpq(1,3) mpq(2,3) mpq(0,1)]
 [mpq(4,3) mpq(5,2) mpq(1,1)]])


PLUSolver(lu=FractionArray(data=[[mpq(3,1) mpq(1,1) mpq(0,1)]
 [mpq(1,3) mpq(2,3) mpq(0,1)]
 [mpq(4,3) mpq(5,2) mpq(1,1)]]), perm=array([0, 1, 2]), _perm_trans=None)

In [11]:
y = solver.btran(c[base])
y

FractionArray(data=[mpq(7,2) mpq(17,2) mpq(0,1)])

In [12]:
p = solver.ftran(b)
p

FractionArray(data=[mpq(54,1) mpq(63,1) mpq(15,1)])

In [13]:
def compute_objective(
    c: fa.FractionArray,
    y: fa.FractionArray,
    table: fas.FractionCSCMatrix,
    nonbasic: np.ndarray,
):
    if np.issubdtype(nonbasic.dtype, bool):
        nonbasic = np.where(nonbasic)[0]
    elif not np.issubdtype(nonbasic.dtype, np.integer):
        raise NotImplementedError("Invalid type {}".format(nonbasic.dtype))

    # TODO: empty?
    tmp = fa.zeros(len(nonbasic))
    for i, col in enumerate(nonbasic):
        start, end = table.indptr[col], table.indptr[col + 1]
        tmp[i] = sum(
            frac * y[row]
            for row, frac in zip(table.indices[start:end], table.data[start:end])
        )
    
    return c[nonbasic] - tmp

In [14]:
obj = compute_objective(c, y, A, ~base)
obj

FractionArray(data=[mpq(-5,2) mpq(3,2) mpq(-7,2) mpq(-17,2)])

In [15]:
entering = np.where(~base)[0][obj.argmax()]
entering

3

In [16]:
# Find d = B^-1 a
# where
#     a - is the entering column (A_N[..., entering])

In [17]:
a = csc_select_columns(A, [entering])[:, 0]
a

FractionArray(data=[mpq(2,1) mpq(1,1) mpq(4,1)])

In [18]:
d = solver.ftran(a)
d

FractionArray(data=[mpq(1,2) mpq(1,2) mpq(1,2)])

In [19]:
valid = np.where(d > 0)[0]

bounds = p[valid] / d[valid]
arg_min_bound = bounds.argmin()

min_bound = bounds[arg_min_bound]
leaving_rel = valid[arg_min_bound]
leaving = np.where(base)[0][leaving_rel]

leaving, leaving_rel, min_bound

(6, 2, mpq(30,1))

In [20]:
p_new = p - min_bound * d 
p_new[leaving_rel] = min_bound
p_new

FractionArray(data=[mpq(39,1) mpq(48,1) mpq(30,1)])

In [21]:
base[entering] = True
base[leaving] = False
base

array([ True, False,  True,  True, False, False, False])

## First iteration

In [22]:
base = base = np.zeros(A.shape[1], dtype=bool)
base[[4, 5,6]] = True
print("Base:", base)

base_ind = np.where(base)[0]
print("Basic indices:", base_ind)

nonbase_ind = np.where(~base)[0]
print("Nonbasic indices:", nonbase_ind)

Base: [False False False False  True  True  True]
Basic indices: [4 5 6]
Nonbasic indices: [0 1 2 3]


In [23]:
# Initialize Eta file
B = csc_select_columns(A, base_ind)
perm, lu = lu_factor(B)
solver = PLUSolver(lu=lu, perm=perm)
print(solver)

eta_file = EtaFile()
eta_file.extend(solver)
eta_file

PLUSolver(lu=FractionArray(data=[[mpq(1,1) mpq(0,1) mpq(0,1)]
 [mpq(0,1) mpq(1,1) mpq(0,1)]
 [mpq(0,1) mpq(0,1) mpq(1,1)]]), perm=array([0, 1, 2]), _perm_trans=None)


EtaFile(file=[PLUSolver(lu=FractionArray(data=[[mpq(1,1) mpq(0,1) mpq(0,1)]
 [mpq(0,1) mpq(1,1) mpq(0,1)]
 [mpq(0,1) mpq(0,1) mpq(1,1)]]), perm=array([0, 1, 2]), _perm_trans=None)])

In [24]:
y = eta_file.btran(c[base_ind])
y

FractionArray(data=[mpq(0,1) mpq(0,1) mpq(0,1)])

In [25]:
obj = compute_objective(c, y, A, nonbase_ind)
obj

FractionArray(data=[mpq(19,1) mpq(13,1) mpq(12,1) mpq(17,1)])

In [26]:
p = eta_file.ftran(b)
p

FractionArray(data=[mpq(225,1) mpq(117,1) mpq(420,1)])

In [27]:
entering_ind = obj.argmin()
entering = nonbase_ind[entering_ind]
entering_ind, entering

(2, 2)

In [28]:
a = csc_select_columns(A, [entering])[:, 0]
a

FractionArray(data=[mpq(1,1) mpq(1,1) mpq(3,1)])

In [29]:
d = eta_file.ftran(a)
d

FractionArray(data=[mpq(1,1) mpq(1,1) mpq(3,1)])

In [30]:
valid = np.where(d > 0)[0]

bounds = p[valid] / d[valid]
arg_min_bound = bounds.argmin()

min_bound = bounds[arg_min_bound]
leaving_ind = valid[arg_min_bound]

leaving = base_ind[leaving_ind]
leaving_ind, leaving

(1, 5)

In [31]:
# Change basis
base[entering] = True
base[leaving] = False
print(base)

base_ind[leaving_ind] = entering
nonbase_ind[entering_ind] = leaving
print(base_ind)
print(nonbase_ind)

new_p = p.copy()
print("new", min_bound * d[valid])
new_p[valid] = p[valid] - min_bound *  d[valid]
new_p[leaving_ind] = min_bound
print(new_p)

p = new_p

[False False  True False  True False  True]
[4 2 6]
[0 1 5 3]
new FractionArray(data=[mpq(117,1) mpq(117,1) mpq(351,1)])
FractionArray(data=[mpq(108,1) mpq(117,1) mpq(69,1)])


In [32]:
# Update eta file
eta = EtaSolver(index=leaving_ind, column=d)
eta_file.extend(eta)

## Second iteration

In [33]:
y = eta_file.btran(c[base_ind])
y

FractionArray(data=[mpq(0,1) mpq(12,1) mpq(0,1)])

In [34]:
obj = compute_objective(c, y, A, nonbase_ind)
obj

FractionArray(data=[mpq(7,1) mpq(1,1) mpq(-12,1) mpq(5,1)])

In [35]:
nonbase_ind
entering_ind = 0
entering = 0
entering_ind, entering

(0, 0)

In [36]:
a = csc_select_columns(A, [entering])[:, 0]
a

FractionArray(data=[mpq(3,1) mpq(1,1) mpq(4,1)])

In [37]:
d = eta_file.ftran(a)
d

FractionArray(data=[mpq(2,1) mpq(1,1) mpq(1,1)])

In [38]:
valid = np.where(d > 0)[0]

bounds = p[valid] / d[valid]
arg_min_bound = bounds.argmin()

min_bound = bounds[arg_min_bound]
leaving_ind = valid[arg_min_bound]

leaving = base_ind[leaving_ind]
leaving_ind, leaving

(0, 4)

In [39]:
# Change basis
base[entering] = True
base[leaving] = False
print(base)

base_ind[leaving_ind] = entering
nonbase_ind[entering_ind] = leaving
print(base_ind)
print(nonbase_ind)

new_p = p.copy()
new_p[valid] = p[valid] - min_bound *  d[valid]
new_p[leaving_ind] = min_bound
print(new_p)

p = new_p

[ True False  True False False False  True]
[0 2 6]
[4 1 5 3]
FractionArray(data=[mpq(54,1) mpq(63,1) mpq(15,1)])


In [40]:
# Update eta file
eta = EtaSolver(index=leaving_ind, column=d)
eta_file.extend(eta)
eta_file

EtaFile(file=[PLUSolver(lu=FractionArray(data=[[mpq(1,1) mpq(0,1) mpq(0,1)]
 [mpq(0,1) mpq(1,1) mpq(0,1)]
 [mpq(0,1) mpq(0,1) mpq(1,1)]]), perm=array([0, 1, 2]), _perm_trans=array([0, 1, 2])), EtaSolver(index=1, column=FractionArray(data=[mpq(1,1) mpq(1,1) mpq(3,1)])), EtaSolver(index=0, column=FractionArray(data=[mpq(2,1) mpq(1,1) mpq(1,1)]))])

## Third iteration

In [41]:
y = eta_file.btran(c[base_ind])
print("y:", y)

# Compute objective function
obj = compute_objective(c, y, A, nonbase_ind)
print("obj:", obj)

# Determine entering
valid_entering = np.where(obj > 0)[0]
entering_ind = valid_entering[obj[valid_entering].argmax()]
entering = nonbase_ind[entering_ind]
print("entering_ind", entering_ind, "entering", entering)

# Determine entering column
a = csc_select_columns(A, [entering])[:, 0]
print("a", a)

d = eta_file.ftran(a)
print("d", d)

# Determine leaving
valid = np.where(d > 0)[0]

bounds = p[valid] / d[valid]
arg_min_bound = bounds.argmin()

min_bound = bounds[arg_min_bound]
leaving_ind = valid[arg_min_bound]

leaving = base_ind[leaving_ind]
print("leaving_ind", leaving_ind, "leaving", leaving)

# Change basis
base[entering] = True
base[leaving] = False
print(base)

base_ind[leaving_ind] = entering
nonbase_ind[entering_ind] = leaving
print("base_ind", base_ind)
print("nonbase_ind", nonbase_ind)

new_p = p.copy()
new_p[valid] = p[valid] - min_bound *  d[valid]
new_p[leaving_ind] = min_bound
print("p", new_p)

p = new_p

# Update eta file
eta = EtaSolver(index=leaving_ind, column=d)
eta_file.extend(eta)
eta_file

y: FractionArray(data=[mpq(7,2) mpq(17,2) mpq(0,1)])
obj: FractionArray(data=[mpq(-7,2) mpq(-5,2) mpq(-17,2) mpq(3,2)])
entering_ind 3 entering 3
a FractionArray(data=[mpq(2,1) mpq(1,1) mpq(4,1)])
d FractionArray(data=[mpq(1,2) mpq(1,2) mpq(1,2)])
leaving_ind 2 leaving 6
[ True False  True  True False False False]
base_ind [0 2 3]
nonbase_ind [4 1 5 6]
p FractionArray(data=[mpq(39,1) mpq(48,1) mpq(30,1)])


EtaFile(file=[PLUSolver(lu=FractionArray(data=[[mpq(1,1) mpq(0,1) mpq(0,1)]
 [mpq(0,1) mpq(1,1) mpq(0,1)]
 [mpq(0,1) mpq(0,1) mpq(1,1)]]), perm=array([0, 1, 2]), _perm_trans=array([0, 1, 2])), EtaSolver(index=1, column=FractionArray(data=[mpq(1,1) mpq(1,1) mpq(3,1)])), EtaSolver(index=0, column=FractionArray(data=[mpq(2,1) mpq(1,1) mpq(1,1)])), EtaSolver(index=2, column=FractionArray(data=[mpq(1,2) mpq(1,2) mpq(1,2)]))])

## Fourth iteration

In [42]:
y = eta_file.btran(c[base_ind])
print("y:", y)

# Compute objective function
obj = compute_objective(c, y, A, nonbase_ind)
print("obj:", obj)

y: FractionArray(data=[mpq(2,1) mpq(1,1) mpq(3,1)])
obj: FractionArray(data=[mpq(-2,1) mpq(-1,1) mpq(-1,1) mpq(-3,1)])


## Everything together



In [43]:
from mcf_simplex_analyzer.simplex import RevisedSimplex, LPModel

model = LPModel(name="example")
x = [ model.new_variable(name=f"x_{i}") for i in range(4)]
print("variables:", x)

objective_function = 19 * x[0] + 13 * x[1] + 12 * x[2] + 17 * x[3]
print("objective_function", objective_function)

c1 = 3 * x[0] + 2 * x[1] + x[2] + 2 * x[3] <= 225
c2 = x[0] + x[1] + x[2] + x[3] <= 117
c3 = 4 * x[0] + 3 * x[1] + 3 * x[2] + 4 * x[3] <= 420

model.constraints = [c1, c2, c3]
model.objective_function = objective_function
model

simplex = RevisedSimplex.instantiate(model)
simplex

simplex.solve()

08:47:59 INFO:Starting the revised simplex algorithm.
08:47:59 DEBUG:refactorization_period=25
08:47:59 INFO:Refactorizing base...
08:47:59 INFO:Base refactorized. Time: 0.0010863800052902661
08:47:59 INFO:
Iteration: 0
08:47:59 DEBUG:y= FractionArray(data=[mpq(0,1) mpq(0,1) mpq(0,1)])
08:47:59 DEBUG:nonbasic_objective= FractionArray(data=[mpq(19,1) mpq(13,1) mpq(12,1) mpq(17,1)])
08:47:59 DEBUG:entering_index= 0
08:47:59 DEBUG:Entering= 0
08:47:59 DEBUG:Entering column= FractionArray(data=[mpq(3,1) mpq(1,1) mpq(4,1)])
08:47:59 DEBUG:d= FractionArray(data=[mpq(3,1) mpq(1,1) mpq(4,1)])
08:47:59 DEBUG:leaving_index= 0
08:47:59 DEBUG:leaving= 4
08:47:59 DEBUG:min_bound= 75
08:47:59 DEBUG:Updating base
08:47:59 DEBUG:base= [ True False False False False  True  True]
08:47:59 DEBUG:base_indices= [0 5 6]
08:47:59 DEBUG:nonbase_indices= [4 1 2 3]
08:47:59 INFO:base values: FractionArray(data=[mpq(75,1) mpq(42,1) mpq(120,1)])
08:47:59 INFO:optimum: 1425
08:47:59 DEBUG:Solution for the current 

variables: [Variable(index=0, name='x_0'), Variable(index=1, name='x_1'), Variable(index=2, name='x_2'), Variable(index=3, name='x_3')]
objective_function LinearCombination(combination=[FactorVar(factor=19, var=Variable(index=0, name='x_0')), FactorVar(factor=13, var=Variable(index=1, name='x_1')), FactorVar(factor=12, var=Variable(index=2, name='x_2')), FactorVar(factor=17, var=Variable(index=3, name='x_3'))])
self.base  [False False False False  True  True  True]
0
1
2
3


LPResult(status='success', value=mpq(1827,1))

In [44]:
isinstance(simplex.table.data, np.ndarray) and simplex.table.data.dtype

False

In [45]:
model = LPModel(name="problem_four_infeasible")
x = [model.new_variable(name=f"x_{i}") for i in range(2)]
slack = [model.new_variable(name=f"slack_{i}") for i in range(3)]

model.objective_function = 3 * x[0] + x[1]
model.constraints = [
    x[0] - x[1] + slack[0] == -1,
    -x[0] - x[1] + slack[1] == -3,
    2 * x[0] + x[1] + slack[2] == 2,
]

simplex = RevisedSimplex.instantiate(model)
ans = simplex.solve()

assert ans.status == "infeasible"

08:47:59 INFO:Starting the revised simplex algorithm.
08:47:59 DEBUG:refactorization_period=25
08:47:59 INFO:Refactorizing base...
08:47:59 INFO:Base refactorized. Time: 0.003522774997691158


self.base  [False False False False False  True  True  True]


08:47:59 INFO:
Iteration: 0
08:47:59 DEBUG:y= FractionArray(data=[mpq(-1,1) mpq(-1,1) mpq(-1,1)])
08:47:59 DEBUG:nonbasic_objective= FractionArray(data=[mpq(2,1) mpq(3,1) mpq(-1,1) mpq(-1,1) mpq(1,1)])
08:47:59 DEBUG:entering_index= 1
08:47:59 DEBUG:Entering= 1
08:47:59 DEBUG:Entering column= FractionArray(data=[mpq(1,1) mpq(1,1) mpq(1,1)])
08:47:59 DEBUG:d= FractionArray(data=[mpq(1,1) mpq(1,1) mpq(1,1)])
08:47:59 DEBUG:leaving_index= 0
08:47:59 DEBUG:leaving= 5
08:47:59 DEBUG:min_bound= 1
08:47:59 DEBUG:Updating base
08:47:59 DEBUG:base= [False  True False False False False  True  True]
08:47:59 DEBUG:base_indices= [1 6 7]
08:47:59 DEBUG:nonbase_indices= [0 5 2 3 4]


0


08:47:59 INFO:base values: FractionArray(data=[mpq(1,1) mpq(2,1) mpq(1,1)])
08:47:59 INFO:optimum: -3
08:47:59 DEBUG:Solution for the current basis: FractionArray(data=[mpq(1,1) mpq(2,1) mpq(1,1)]) 
08:47:59 DEBUG:
Iteration: 1
08:47:59 DEBUG:y= FractionArray(data=[mpq(2,1) mpq(-1,1) mpq(-1,1)])
08:47:59 DEBUG:nonbasic_objective= FractionArray(data=[mpq(5,1) mpq(-3,1) mpq(2,1) mpq(-1,1) mpq(1,1)])
08:47:59 DEBUG:entering_index= 0
08:47:59 DEBUG:Entering= 0
08:47:59 DEBUG:Entering column= FractionArray(data=[mpq(-1,1) mpq(1,1) mpq(2,1)])
08:47:59 DEBUG:d= FractionArray(data=[mpq(-1,1) mpq(2,1) mpq(3,1)])
08:47:59 DEBUG:leaving_index= 2
08:47:59 DEBUG:leaving= 7
08:47:59 DEBUG:min_bound= 1/3
08:47:59 DEBUG:Updating base
08:47:59 DEBUG:base= [ True  True False False False False  True False]
08:47:59 DEBUG:base_indices= [1 6 0]
08:47:59 DEBUG:nonbase_indices= [7 5 2 3 4]
08:47:59 INFO:base values: FractionArray(data=[mpq(4,3) mpq(4,3) mpq(1,3)])


1


08:47:59 INFO:optimum: -4/3
08:47:59 DEBUG:Solution for the current basis: FractionArray(data=[mpq(4,3) mpq(4,3) mpq(1,3)]) 
08:47:59 DEBUG:
Iteration: 2
08:47:59 DEBUG:y= FractionArray(data=[mpq(1,3) mpq(-1,1) mpq(2,3)])
08:47:59 DEBUG:nonbasic_objective= FractionArray(data=[mpq(-5,3) mpq(-4,3) mpq(1,3) mpq(-1,1) mpq(-2,3)])
08:47:59 DEBUG:entering_index= 2
08:47:59 DEBUG:Entering= 2
08:47:59 DEBUG:Entering column= FractionArray(data=[mpq(-1,1) mpq(0,1) mpq(0,1)])
08:47:59 DEBUG:d= FractionArray(data=[mpq(-2,3) mpq(1,3) mpq(1,3)])
08:47:59 DEBUG:leaving_index= 2
08:47:59 DEBUG:leaving= 0
08:47:59 DEBUG:min_bound= 1
08:47:59 DEBUG:Updating base
08:47:59 DEBUG:base= [False  True  True False False False  True False]
08:47:59 DEBUG:base_indices= [1 6 2]
08:47:59 DEBUG:nonbase_indices= [7 5 0 3 4]
08:47:59 INFO:base values: FractionArray(data=[mpq(2,1) mpq(1,1) mpq(1,1)])
08:47:59 INFO:optimum: -1
08:47:59 DEBUG:Solution for the current basis: FractionArray(data=[mpq(2,1) mpq(1,1) mpq(1,1)

2
3
